# Retrieval Sweep Results Analysis
This notebook loads CSV results generated by `scripts/sweep_retrieval.py`,
compares performance by configuration, and selects top combinations.


In [ ]:

import pandas as pd
from pathlib import Path
from caas_jupyter_tools import display_dataframe_to_user

results_dir = Path('/mnt/data/results')
summary_csv = list(results_dir.glob('*.csv'))
summary_csv


In [ ]:

import pandas as pd
from pathlib import Path

# Specify file paths directly
summary_path = Path('/mnt/data/results/sweep_run.csv')
per_query_path = Path('/mnt/data/results/sweep_run_per_query.csv')

summary = pd.read_csv(summary_path)
per_query = pd.read_csv(per_query_path)

summary.head(), per_query.head()


## Top-N Combination Selection

In [ ]:

# Simple score: hit_rate 0.6, kw_hit_rate 0.3, - avg_latency_s 0.1 weighted sum
summary = summary.copy()
summary['score'] = 0.6*summary['hit_rate'] + 0.3*summary['kw_hit_rate'] - 0.1*summary['avg_latency_s']
top = summary.sort_values('score', ascending=False).head(10)
top


## Visualization (matplotlib, following unspecified color rules)

In [ ]:

import matplotlib.pyplot as plt

plt.figure()
plt.scatter(summary['avg_latency_s'], summary['hit_rate'])
plt.xlabel('Avg Latency (s)')
plt.ylabel('Hit Rate')
plt.title('Latency vs Hit Rate')
plt.show()


## Performance by Intent/Query Type (Optional: when label column is added to per_query CSV)

In [ ]:

# Example: If per_query['intent'] exists, you can see group averages.
if 'intent' in per_query.columns:
    print(per_query.groupby('intent')[['hit','kw_hit','coverage','latency_s']].mean())
else:
    print('No intent column in per_query.csv. If you want this, add intent to eval_set and pass it down in the script.')
